In [1]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

##images:

from skimage.transform import resize
from skimage.io import imread

import os #os is a library deals with files and paths
import matplotlib.pyplot as plt

import pickle

import warnings

warnings.simplefilter ('ignore')


In [2]:
datadir = 'imageclassification/Images/'
categories = ['ChickenHead','ElephantHead']

flat_data_arr = []
target_arr = []

for i in categories:
    print ('loading...category:',i)
    path = os.path.join (datadir, i)
    for img in os.listdir (path):
        img_array = imread (os.path.join (path,img))
        img_resized = resize (img_array,(150,150,3))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(categories.index(i))
    print ('loaded category', i, 'successfully')

loading...category: ChickenHead
loaded category ChickenHead successfully
loading...category: ElephantHead
loaded category ElephantHead successfully


In [3]:
flat_data = np.array(flat_data_arr)
target = np.array (target_arr)

df = pd.DataFrame (flat_data)
df['Target'] = target

df.head()

,0,1,2,3,4,5,6,7,8,9,...,67491,67492,67493,67494,67495,67496,67497,67498,67499,Target
0,0.258824,0.258824,0.219608,0.231373,0.231373,0.192157,0.168627,0.168627,0.129412,0.180392,...,0.321569,0.321569,0.290196,0.301961,0.301961,0.270588,0.290196,0.290196,0.258824,0
1,0.121569,0.227451,0.105882,0.129412,0.227451,0.113725,0.176471,0.254902,0.149020,0.219608,...,0.784314,0.670588,0.513725,0.772549,0.658824,0.501961,0.768627,0.654902,0.498039,0
2,0.415686,0.698039,0.258824,0.384314,0.666667,0.235294,0.349020,0.619608,0.203922,0.321569,...,0.423529,0.615686,0.266667,0.435294,0.627451,0.270588,0.380392,0.572549,0.215686,0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,0.784314,0.729412,0.623529,0.776471,0.721569,0.615686,0.737255,0.682353,0.576471,0.690196,...,0.717647,0.643137,0.517647,0.725490,0.650980,0.525490,0.721569,0.647059,0.521569,0


In [4]:
X = df.values[:, :-1]
Y = df.values[:,-1]

xtrain, xtest, ytrain, ytest = train_test_split (X, Y, test_size=0.2, random_state=47)

In [54]:
param_grid = { 'C': [10, 100], 'gamma': [0.0001], 'kernel':['rbf']}
svc = SVC()

model = GridSearchCV (svc, param_grid)
model.fit (xtrain,ytrain)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [10, 100], 'gamma': [0.0001], 'kernel': ['rbf']})

In [55]:
print (model.best_params_)
ypred_train = model.predict (xtrain)
ypred_test =model.predict (xtest)

print('Train Accuracy:', accuracy_score (ypred_train, ytrain))
print('Test Accuracy:', accuracy_score (ypred_test, ytest))


{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
Train Accuracy: 1.0
Test Accuracy: 0.9


In [56]:
pickle.dump (model, open('supportvectormachineModel.pkl', 'wb'))

In [57]:
model = pickle.load (open('supportvectormachineModel.pkl', 'rb'))

In [58]:
filename = 'imageclassification/Images/elephent1.jpg'
img_array = imread (filename)
img_resized = resize(img_array, (150,150,3))
test_data =np.array (img_resized.flatten())
test_data =test_data.reshape(1,-1)
print (test_data.shape)

(1, 67500)


In [59]:
p = model.predict (test_data)
if p==0: print ('Chicken')
else: print ('Elephant')

Elephant


In [60]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
for k in range (5, 10, 1):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(xtrain, ytrain)
    y_pred = knn.predict (xtest)
    a =accuracy_score (y_pred, ytest)
    print ("For k=", k, "Accuracy =", a)

For k= 5 Accuracy = 0.825
For k= 6 Accuracy = 0.875
For k= 7 Accuracy = 0.825
For k= 8 Accuracy = 0.85
For k= 9 Accuracy = 0.8


In [61]:
pickle.dump (model, open('KNeighborsClassifierModel.pkl', 'wb'))

In [62]:
model = pickle.load (open('KNeighborsClassifierModel.pkl', 'rb'))

In [63]:
filename = 'imageclassification/Images/elephent2.jpg'
img_array = imread (filename)
img_resized = resize(img_array, (150,150,3))
test_data =np.array (img_resized.flatten())
test_data =test_data.reshape(1,-1)
print (test_data.shape)

(1, 67500)


In [64]:
p = model.predict (test_data)
if p==0: print ('Chicken')
else: print ('Elephant')

Elephant


In [65]:
from sklearn.ensemble import RandomForestClassifier
maxAccuracy = -1
best_estimators =25
for n in range (25, 200, 1):
    rf=RandomForestClassifier (n_estimators=n)
    rf.fit(xtrain, ytrain)
    y_pred3 = rf.predict(xtest)
    a3 = accuracy_score (y_pred3, ytest)
    if a3 > maxAccuracy:
        maxAccuracy =a3
        best_estimators = n
print ('Accuracy =', maxAccuracy, 'for n=', best_estimators)

Accuracy = 0.975 for n= 36


In [66]:
pickle.dump (model, open('RandomForestClassifierModel.pkl', 'wb'))

In [67]:
model = pickle.load (open('RandomForestClassifierModel.pkl', 'rb'))

In [68]:
filename = 'imageclassification/Images/chicken1.png'
img_array = imread (filename)
img_resized = resize(img_array, (150,150,3))
test_data =np.array (img_resized.flatten())
test_data =test_data.reshape(1,-1)
print (test_data.shape)

(1, 67500)


In [69]:
p = model.predict (test_data)
if p==0: print ('Chicken')
else: print ('Elephant')

Chicken


In [70]:
ytrain[ytrain == 0] =-1
ytest[ytest == 0] = -1

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

maxAccuracy = -1
best_estimators = 25

for n in range(25, 200, 1):
    ab = AdaBoostClassifier(n_estimators=n)
    ab.fit(xtrain, ytrain)
    y_pred4 = ab.predict(xtest)
    a4 = accuracy_score(y_pred4, ytest)
    if a4 > maxAccuracy:
        maxAccuracy = a4
        best_estimators = n

print('Accuracy =', maxAccuracy, 'for n=', best_estimators)


In [72]:
pickle.dump (model, open('AdaBoostClassifierModel.pkl', 'wb'))

In [73]:
model = pickle.load (open('AdaBoostClassifierModel.pkl', 'rb'))

In [ ]:
filename = 'imageclassification/Images/chicken2.png'
img_array = imread (filename)
img_resized = resize(img_array, (150,150,3))
test_data =np.array (img_resized.flatten())
test_data =test_data.reshape(1,-1)
print (test_data.shape)

In [76]:
p = model.predict (test_data)
if p==0: print ('Chicken')
else: print ('Elephant')

Chicken


In [78]:
from sklearn.neural_network import MLPClassifier

r = MLPClassifier(hidden_layer_sizes=(2,), activation='relu', solver='adam', max_iter=500)
r.fit(xtrain, ytrain)
r.predict(xtest)
r.score(xtest, ytest)


0.4

In [81]:
pickle.dump (model, open('neuralnetworkModel.pkl', 'wb'))

In [82]:
model = pickle.load (open('neuralnetworkModel.pkl', 'rb'))

In [84]:
filename = 'imageclassification/Images/chicken3.jpg'
img_array = imread (filename)
img_resized = resize(img_array, (150,150,3))
test_data =np.array (img_resized.flatten())
test_data =test_data.reshape(1,-1)
print (test_data.shape)

(1, 67500)


In [85]:
p = model.predict (test_data)
if p==0: print ('Chicken')
else: print ('Elephant')

Chicken
